# **INTRODUCTIONS**

---

Used for exceptional processes.

The code is separated into multiple sections:

1. **[VACCINE_COVERAGE_PER_AGE_GROUP_ESTIMATED_ARCHIVED_20220908](#VACCINE_COVERAGE_PER_AGE_GROUP_ESTIMATED_ARCHIVED_20220908)**
2. **[xxxx](#xxxx)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/utils/conversions.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

# **VACCINE_COVERAGE_PER_AGE_GROUP_ESTIMATED_ARCHIVED_20220908**

---

table

In [ ]:
-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSARCHIVE].[RIVM_VACCINATION_SHOT_COVERAGES_PER_CAMPAIGN_NL_ARCHIVED_20220908]'))
CREATE TABLE [VWSARCHIVE].[RIVM_VACCINATION_SHOT_COVERAGES_PER_CAMPAIGN_NL_ARCHIVED_20220908] (
	[ID] BIGINT,
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
    [DATE_OF_STATISTICS] DATETIME,
    [DATE_OF_REPORT] DATETIME,
	[AGE_GROUP] VARCHAR(50) NULL,
    [BIRTH_YEAR] VARCHAR(50),
    [PERCENTAGE_PARTLY] FLOAT,
    [PERCENTAGE_COMPLETED] FLOAT
);
GO


stored procedure

In [ ]:

CREATE OR ALTER PROCEDURE [dbo].[SP_RIVM_VACCINATION_SHOT_COVERAGES_PER_CAMPAIGN_NL_ARCHIVED_20220908] AS
BEGIN
    TRUNCATE TABLE [VWSARCHIVE].[RIVM_VACCINATION_SHOT_COVERAGES_PER_CAMPAIGN_NL_ARCHIVED_20220908]

    INSERT INTO [VWSARCHIVE].[RIVM_VACCINATION_SHOT_COVERAGES_PER_CAMPAIGN_NL_ARCHIVED_20220908] (
        [ID],
        [DATE_LAST_INSERTED],
        [DATE_OF_STATISTICS],
        [DATE_OF_REPORT],
        [AGE_GROUP],
        [BIRTH_YEAR],
        [PERCENTAGE_PARTLY],
        [PERCENTAGE_COMPLETED]
    )
    SELECT 
        [ID],
        [DATE_LAST_INSERTED],
        [DATE_OF_STATISTICS],
        [DATE_OF_REPORT],
        [AGE_GROUP],
        [BIRTH_YEAR],
        [PERCENTAGE_PARTLY],
        [PERCENTAGE_COMPLETED]
    FROM VWSDEST.RIVM_VACCINATION_SHOT_COVERAGES_NL
    WHERE CAST([DATE_LAST_INSERTED] as Date) = CAST('2022-09-08' as Date)
END


View

In [ ]:
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_VACCINATION_SHOT_COVERAGES_NL_ARCHIVED_20220908] AS
    WITH CTE AS (
        SELECT
            [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX],
            [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_STATISTICS]) AS [DATE_UNIX],
            [DATE_OF_REPORT],
            [AGE_GROUP],
            REPLACE([BIRTH_YEAR], '<=', '-') AS [BIRTH_YEAR],
            CAST([PERCENTAGE_PARTLY] AS [NUMERIC](10,1)) AS [VACCINATION_COVERAGE_PARTLY],
            CAST([PERCENTAGE_COMPLETED] AS [NUMERIC](10,1)) AS [VACCINATION_COVERAGE_COMPLETED]
        FROM [VWSARCHIVE].[RIVM_VACCINATION_SHOT_COVERAGES_PER_CAMPAIGN_NL_ARCHIVED_20220908] WITH(NOLOCK)
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSARCHIVE].[RIVM_VACCINATION_SHOT_COVERAGES_PER_CAMPAIGN_NL_ARCHIVED_20220908] WITH(NOLOCK))
    ),
    COVERAGE_12_PLUS AS (
        SELECT
            [DATE_UNIX],
            [DATE_OF_INSERTION_UNIX],
            [DATE_OF_REPORT],
            [BIRTH_YEAR],
            [VACCINATION_COVERAGE_PARTLY],
            [VACCINATION_COVERAGE_COMPLETED]
        FROM CTE
        WHERE [DATE_UNIX] = (SELECT MAX([DATE_UNIX]) FROM CTE)
            AND [AGE_GROUP] = '12+'
    ),
    COVERAGE_18_PLUS AS (
        SELECT
            [DATE_UNIX],
            [DATE_OF_INSERTION_UNIX],
            [DATE_OF_REPORT],
            [BIRTH_YEAR],
            [VACCINATION_COVERAGE_PARTLY],
            [VACCINATION_COVERAGE_COMPLETED]
        FROM CTE
        WHERE [DATE_UNIX] = (SELECT MAX([DATE_UNIX]) FROM CTE)
            AND [AGE_GROUP] = '18+'
    )
    SELECT
        T1.[BIRTH_YEAR] AS [AGE_12_PLUS_BIRTHYEAR],
        T1.[VACCINATION_COVERAGE_PARTLY] AS [AGE_12_PLUS_HAS_ONE_SHOT],
        T1.[VACCINATION_COVERAGE_COMPLETED] AS [AGE_12_PLUS_FULLY_VACCINATED],
        T2.[BIRTH_YEAR] AS [AGE_18_PLUS_BIRTHYEAR],
        T2.[VACCINATION_COVERAGE_PARTLY] AS [AGE_18_PLUS_HAS_ONE_SHOT],
        T2.[VACCINATION_COVERAGE_COMPLETED] AS [AGE_18_PLUS_FULLY_VACCINATED],
        T2.[DATE_UNIX],
        T2.[DATE_OF_INSERTION_UNIX]
    FROM COVERAGE_12_PLUS T1
    INNER JOIN COVERAGE_18_PLUS T2 ON T1.[DATE_OF_REPORT] = T2.[DATE_OF_REPORT]
GO

protos

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_VACCINATION_SHOT_COVERAGES_NL_ARCHIVED_20220908',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_coverage_per_age_group_estimated_archived_20220908',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1,
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO